In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

# Chi-Sqaure for association on variables

In [7]:
expected_counts = np.ones((2, 2)) * (120 / (2 * 2))

# Generate observed counts with potential dependence based on effect_size
observed_counts = np.random.multinomial(
    n=120, 
    pvals=expected_counts.flatten() / 120
)
observed_counts = observed_counts.reshape(2, 2)

In [9]:
observed_counts

array([[31, 27],
       [28, 34]])

In [16]:
perturbation = np.random.normal(0, 100, size=(2, 2))

In [17]:
perturbation

array([[-107.98907598, -201.53847237],
       [  66.54449749,  152.0444155 ]])

In [18]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import seaborn as sns

# Set a random seed for reproducibility
np.random.seed(42)

def simulate_chi_square_test(rows=2, 
                            cols=2, 
                            n_samples=100, 
                            effect_size=0.3, 
                            n_simulations=10000):
    """
    Simulates chi-square tests multiple times and analyzes the results.
    
    Parameters:
    - rows: Number of categories for the first variable
    - cols: Number of categories for the second variable
    - n_samples: Total sample size
    - effect_size: Controls the strength of association between variables
                  (0 = independence, higher values = stronger association)
    - n_simulations: Number of simulations to run
    
    Returns:
    - DataFrame with simulation results
    """
    p_values = []
    chi2_stats = []
    reject_null = []
    
    for _ in range(n_simulations):
        # Create a contingency table with expected counts under independence
        expected_counts = np.ones((rows, cols)) * (n_samples / (rows * cols))
        
        # Generate observed counts with potential dependence based on effect_size
        observed_counts = np.random.multinomial(
            n=n_samples, 
            pvals=expected_counts.flatten() / n_samples
        )
        
        # Add dependency between variables if effect_size > 0
        if effect_size > 0:
            # Reshape to contingency table
            observed_counts = observed_counts.reshape(rows, cols)
            
            # Create a perturbation matrix for dependency
            perturbation = np.random.normal(0, effect_size, size=(rows, cols))
            
            # Apply perturbation while keeping row/column sums constant
            row_sums = observed_counts.sum(axis=1, keepdims=True)
            col_sums = observed_counts.sum(axis=0, keepdims=True)
            
            # Add perturbation while maintaining constraints
            for i in range(3):  # A few iterations to approximately preserve constraints
                observed_counts = observed_counts * (1 + perturbation)
                observed_counts = observed_counts * (row_sums / observed_counts.sum(axis=1, keepdims=True))
                observed_counts = observed_counts * (col_sums / observed_counts.sum(axis=0, keepdims=True))
            
            # Round to integer counts
            observed_counts = np.round(observed_counts).astype(int)
            
            # Adjust to ensure sum equals n_samples
            diff = n_samples - observed_counts.sum()
            if diff != 0:
                observed_counts[0, 0] += diff
        else:
            observed_counts = observed_counts.reshape(rows, cols)
        
        # Calculate expected frequencies based on marginal totals
        row_totals = observed_counts.sum(axis=1, keepdims=True)
        col_totals = observed_counts.sum(axis=0, keepdims=True)
        expected = np.dot(row_totals, col_totals.T) / n_samples
        
        # Calculate chi-square statistic
        chi2_stat = np.sum(((observed_counts - expected) ** 2) / expected)
        
        # Calculate p-value
        df = (rows - 1) * (cols - 1)
        p_value = 1 - stats.chi2.cdf(chi2_stat, df)
        
        p_values.append(p_value)
        chi2_stats.append(chi2_stat)
        reject_null.append(p_value < 0.05)
    
    # Create a DataFrame with results
    results = pd.DataFrame({
        'Chi2-Statistic': chi2_stats,
        'P-Value': p_values,
        'Reject Null': reject_null
    })
    
    # Calculate the percentage of times we rejected the null hypothesis (statistical power)
    power = results['Reject Null'].mean() * 100
    
    # Display summary statistics
    print(f"Number of simulations: {len(results)}")
    print(f"Percentage of tests that rejected the null hypothesis: {power:.2f}%")
    print("\nSummary of Chi-square statistics:")
    print(results['Chi2-Statistic'].describe())
    print("\nSummary of p-values:")
    print(results['P-Value'].describe())

    # Create visualizations
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # Histogram of chi-square statistics
    ax1.hist(results['Chi2-Statistic'], bins=30, edgecolor='black', alpha=0.7)
    critical_value = stats.chi2.ppf(0.95, df)
    ax1.axvline(x=critical_value, color='red', linestyle='--', 
                label=f'Critical value (α=0.05, df={df}): {critical_value:.2f}')
    ax1.set_title('Distribution of Chi-Square Statistics')
    ax1.set_xlabel('Chi-Square Statistic')
    ax1.set_ylabel('Frequency')
    ax1.legend()

    # Histogram of p-values
    ax2.hist(results['P-Value'], bins=30, edgecolor='black', alpha=0.7)
    ax2.axvline(x=0.05, color='red', linestyle='--', label='α = 0.05')
    ax2.set_title('Distribution of p-Values')
    ax2.set_xlabel('p-Value')
    ax2.set_ylabel('Frequency')
    ax2.legend()

    plt.tight_layout()
    plt.show()

    # Example of a single test for demonstration
    print("\nExample of a single Chi-square test:")
    
    # Generate a random contingency table with potential dependence
    expected_counts = np.ones((rows, cols)) * (n_samples / (rows * cols))
    observed_counts = np.random.multinomial(n=n_samples, pvals=expected_counts.flatten() / n_samples).reshape(rows, cols)
    
    # Add dependency
    if effect_size > 0:
        perturbation = np.random.normal(0, effect_size, size=(rows, cols))
        row_sums = observed_counts.sum(axis=1, keepdims=True)
        col_sums = observed_counts.sum(axis=0, keepdims=True)
        
        for i in range(3):
            observed_counts = observed_counts * (1 + perturbation)
            observed_counts = observed_counts * (row_sums / observed_counts.sum(axis=1, keepdims=True))
            observed_counts = observed_counts * (col_sums / observed_counts.sum(axis=0, keepdims=True))
        
        observed_counts = np.round(observed_counts).astype(int)
        diff = n_samples - observed_counts.sum()
        if diff != 0:
            observed_counts[0, 0] += diff
    
    # Calculate expected frequencies
    row_totals = observed_counts.sum(axis=1, keepdims=True)
    col_totals = observed_counts.sum(axis=0, keepdims=True)
    expected = np.dot(row_totals, col_totals.T) / n_samples
    
    # Print the contingency table
    print("Observed counts:")
    print(observed_counts)
    print("\nExpected counts:")
    print(expected.round(2))
    
    # Calculate chi-square statistic
    chi2_stat = np.sum(((observed_counts - expected) ** 2) / expected)
    
    # Calculate p-value
    df = (rows - 1) * (cols - 1)
    p_value = 1 - stats.chi2.cdf(chi2_stat, df)
    
    print(f"\nChi-square statistic: {chi2_stat:.4f}")
    print(f"Degrees of freedom: {df}")
    print(f"p-value: {p_value:.4f}")
    print(f"Reject null hypothesis: {p_value < 0.05}")
    
    # Visualize the single contingency table
    plt.figure(figsize=(10, 6))
    sns.heatmap(observed_counts, annot=True, fmt='d', cmap='YlGnBu')
    plt.title('Observed Contingency Table')
    plt.tight_layout()
    plt.show()
    
    return results


In [21]:
# Example usage:
# No association (independence)
results_null = simulate_chi_square_test(rows=2, cols=2, effect_size=0, n_simulations=1000)

ValueError: shapes (2,1) and (2,1) not aligned: 1 (dim 1) != 2 (dim 0)

In [ ]:


# With association (dependency)
results_alt = simulate_chi_square_test(rows=3, cols=4, effect_size=0.5, n_simulations=1000)

# Chi-Square Goodness of Fit Test Simulation in Python

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd

# Set a random seed for reproducibility
np.random.seed(42)

def simulate_goodness_of_fit_test(categories=4, 
                                 expected_props=None,
                                 observed_props=None, 
                                 n_samples=100, 
                                 deviation=0.1,
                                 n_simulations=10000):
    """
    Simulates chi-square goodness of fit tests multiple times and analyzes the results.
    
    Parameters:
    - categories: Number of categories for the variable
    - expected_props: Expected proportions for each category (default is equal proportions)
    - observed_props: Observed proportions to sample from (default derives from expected + deviation)
    - n_samples: Total sample size
    - deviation: Controls how much observed proportions deviate from expected
                (0 = no deviation, higher values = stronger deviation)
    - n_simulations: Number of simulations to run
    
    Returns:
    - DataFrame with simulation results
    """
    p_values = []
    chi2_stats = []
    reject_null = []
    
    # Set up expected proportions if not provided
    if expected_props is None:
        expected_props = np.ones(categories) / categories
    else:
        expected_props = np.array(expected_props)
        expected_props = expected_props / expected_props.sum()  # Normalize to sum to 1
    
    # Set up observed proportions if not provided
    if observed_props is None:
        if deviation == 0:
            observed_props = expected_props.copy()
        else:
            # Create random deviations from expected proportions
            observed_props = expected_props + np.random.uniform(-deviation, deviation, size=categories)
            # Ensure all proportions are positive
            observed_props = np.maximum(observed_props, 0.01)
            # Normalize to sum to 1
            observed_props = observed_props / observed_props.sum()
    else:
        observed_props = np.array(observed_props)
        observed_props = observed_props / observed_props.sum()  # Normalize to sum to 1
    
    for _ in range(n_simulations):
        # Generate observed counts
        observed_counts = np.random.multinomial(n=n_samples, pvals=observed_props)
        
        # Calculate expected counts based on expected proportions
        expected_counts = n_samples * expected_props
        
        # Calculate chi-square statistic
        chi2_stat = np.sum(((observed_counts - expected_counts) ** 2) / expected_counts)
        
        # Calculate p-value
        df = categories - 1
        p_value = 1 - stats.chi2.cdf(chi2_stat, df)
        
        p_values.append(p_value)
        chi2_stats.append(chi2_stat)
        reject_null.append(p_value < 0.05)
    
    # Create a DataFrame with results
    results = pd.DataFrame({
        'Chi2-Statistic': chi2_stats,
        'P-Value': p_values,
        'Reject Null': reject_null
    })
    
    # Calculate the percentage of times we rejected the null hypothesis (statistical power)
    power = results['Reject Null'].mean() * 100
    
    # Display summary statistics
    print(f"Number of simulations: {len(results)}")
    print(f"Percentage of tests that rejected the null hypothesis: {power:.2f}%")
    print("\nSummary of Chi-square statistics:")
    print(results['Chi2-Statistic'].describe())
    print("\nSummary of p-values:")
    print(results['P-Value'].describe())

    # Create visualizations
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # Histogram of chi-square statistics
    ax1.hist(results['Chi2-Statistic'], bins=30, edgecolor='black', alpha=0.7)
    critical_value = stats.chi2.ppf(0.95, df)
    ax1.axvline(x=critical_value, color='red', linestyle='--', 
                label=f'Critical value (α=0.05, df={df}): {critical_value:.2f}')
    ax1.set_title('Distribution of Chi-Square Statistics')
    ax1.set_xlabel('Chi-Square Statistic')
    ax1.set_ylabel('Frequency')
    ax1.legend()

    # Histogram of p-values
    ax2.hist(results['P-Value'], bins=30, edgecolor='black', alpha=0.7)
    ax2.axvline(x=0.05, color='red', linestyle='--', label='α = 0.05')
    ax2.set_title('Distribution of p-Values')
    ax2.set_xlabel('p-Value')
    ax2.set_ylabel('Frequency')
    ax2.legend()

    plt.tight_layout()
    plt.show()

    # Example of a single test for demonstration
    print("\nExample of a single Chi-square Goodness of Fit test:")
    
    # Generate observed counts for a single test
    observed_counts = np.random.multinomial(n=n_samples, pvals=observed_props)
    expected_counts = n_samples * expected_props
    
    # Print the data
    print("Category\tObserved\tExpected\tO-E\t(O-E)²/E")
    chi2_components = ((observed_counts - expected_counts) ** 2) / expected_counts
    for i in range(categories):
        print(f"{i+1}\t\t{observed_counts[i]}\t\t{expected_counts[i]:.1f}\t\t{observed_counts[i]-expected_counts[i]:.1f}\t{chi2_components[i]:.3f}")
    
    # Calculate chi-square statistic
    chi2_stat = np.sum(chi2_components)
    
    # Calculate p-value
    df = categories - 1
    p_value = 1 - stats.chi2.cdf(chi2_stat, df)
    
    print(f"\nChi-square statistic: {chi2_stat:.4f}")
    print(f"Degrees of freedom: {df}")
    print(f"p-value: {p_value:.4f}")
    print(f"Reject null hypothesis: {p_value < 0.05}")
    
    # Visualize the expected vs observed
    fig, ax = plt.subplots(figsize=(10, 6))
    index = np.arange(categories)
    bar_width = 0.35
    
    ax.bar(index, observed_counts, bar_width, label='Observed', color='skyblue')
    ax.bar(index + bar_width, expected_counts, bar_width, label='Expected', color='lightgreen')
    
    ax.set_xlabel('Category')
    ax.set_ylabel('Frequency')
    ax.set_title('Observed vs. Expected Frequencies')
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels([f'Cat {i+1}' for i in range(categories)])
    ax.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Show proportions comparison
    plt.figure(figsize=(10, 6))
    plt.bar(index, expected_props, bar_width, label='Expected Proportions', color='lightgreen')
    plt.bar(index + bar_width, observed_props, bar_width, label='Population Proportions', color='salmon')
    
    plt.xlabel('Category')
    plt.ylabel('Proportion')
    plt.title('Expected vs. Population Proportions')
    plt.xticks(index + bar_width / 2, [f'Cat {i+1}' for i in range(categories)])
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    return results
